In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Masking, TimeDistributed, Flatten, BatchNormalization
from tensorflow.keras import optimizers
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping

# Load dataset
file_path = 'data/statcast.csv'
data = pd.read_csv(file_path)

# Drop unnecessary columns
data = data.drop(columns=['last_name, first_name'])

# Fill missing values with 0
data = data.fillna(0)

if 'pitch_hand' in data:
    data = pd.get_dummies(data, columns=['pitch_hand'], drop_first=True)

# Extract player_ids that exist in all years 2019, 2020, 2021, 2022, and 2023
data_2019 = data[data['year'] == 2019]
data_2020 = data[data['year'] == 2020]
data_2021 = data[data['year'] == 2021]
data_2022 = data[data['year'] == 2022]
data_2023 = data[data['year'] == 2023]

player_ids_2019 = set(data_2019['player_id'].unique())
player_ids_2020 = set(data_2020['player_id'].unique())
player_ids_2021 = set(data_2021['player_id'].unique())
player_ids_2022 = set(data_2022['player_id'].unique())
player_ids_2023 = set(data_2023['player_id'].unique())

common_player_ids = player_ids_2019 & player_ids_2020 & player_ids_2021 & player_ids_2022 & player_ids_2023

# Extract data for common player_ids
common_data = data[data['player_id'].isin(common_player_ids)]

# Extract data for the years 2019, 2020, 2021, and 2022
final = common_data[common_data['year'].isin([2019, 2020, 2021, 2022])]
final = final.sort_values(by=['player_id', 'year'])

# Select necessary columns (excluding year)
features = [col for col in final.columns if col not in ['player_id', 'year', 'p_era']]
target = 'p_era'

# Separate independent and dependent variables
X = final[features].values
y = final[target].values

# Data scaling
scaler_X = MinMaxScaler()
scaler_y = MinMaxScaler()

X_scaled = scaler_X.fit_transform(X)
y_scaled = scaler_y.fit_transform(y.reshape(-1, 1))

# Convert to time series data
def create_sequences(X, y, seq_length):
    X_seq, y_seq = [], []
    for i in range(len(X)):
        seq_x = X[max(0, i - seq_length + 1):i + 1]
        seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
        seq_y = y[i]
        X_seq.append(seq_x)
        y_seq.append(seq_y)
    return np.array(X_seq), np.array(y_seq)

seq_length = 4  # Set sequence length
X_seq, y_seq = create_sequences(X_scaled, y_scaled, seq_length)

# Set training data to be the same as the entire data
X_train, y_train = X_seq, y_seq

In [2]:
# LSTM_A

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_A
    model_A = Sequential()
    model_A.add(LSTM(128, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_A.add(LSTM(128, return_sequences=True))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Flatten())
    model_A.add(Dense(1024, activation="relu"))
    model_A.add(Dropout(rate=0.5))
    model_A.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_A.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_A = model_A.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Convert to time series data (use 2022, 2021, 2020, 2019 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_A = model_A.predict(X_2023_seq)

    # Inverse scale
    y_pred_A = scaler_y.inverse_transform(y_pred_scaled_A)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_A = np.sqrt(mean_squared_error(y_test_actual, y_pred_A))
    rmse_list.append(rmse_A)

    # Calculate MAE
    mae_A = mean_absolute_error(y_test_actual, y_pred_A)
    mae_list.append(mae_A)
    
    # Calculate MAPE
    mape_A = mean_absolute_percentage_error(y_test_actual, y_pred_A)
    mape_list.append(mape_A)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_A}, MAE: {mae_A}, MAPE: {mape_A}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
4/4 [==============================] - 18s 2s/step - loss: 0.0639 - accuracy: 0.0050 - val_loss: 0.0376 - val_accuracy: 0.0196
Epoch 2/500
4/4 [==============================] - 0s 57ms/step - loss: 0.0388 - accuracy: 0.0000e+00 - val_loss: 0.0345 - val_accuracy: 0.0196
Epoch 3/500
4/4 [==============================] - 0s 50ms/step - loss: 0.0247 - accuracy: 0.0000e+00 - val_loss: 0.0378 - val_accuracy: 0.0196
Epoch 4/500
4/4 [==============================] - 0s 63ms/step - loss: 0.0258 - accuracy: 0.0000e+00 - val_loss: 0.0335 - val_accuracy: 0.0196
Epoch 5/500
4/4 [==============================] - 0s 60ms/step - loss: 0.0259 - accuracy: 0.0000e+00 - val_loss: 0.0335 - val_accuracy: 0.0196
Epoch 6/500
4/4 [==============================] - 0s 48ms/step - loss: 0.0270 - accuracy: 0.0000e+00 - val_loss: 0.0338 - val_accuracy: 0.0196
Epoch 7/500
4/4 [==============================] - 0s 58ms/step - loss: 0.0228 - accuracy: 0.0000e+00 - val_loss: 0.0327 - val_accuracy: 0.01

In [3]:
# LSTM_B

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_B
    model_B = Sequential()
    model_B.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_B.add(LSTM(16, return_sequences=True))
    model_B.add(LSTM(8, return_sequences=True))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Flatten())
    model_B.add(Dense(512, activation="relu"))
    model_B.add(Dropout(rate=0.5))
    model_B.add(Dense(64, activation="relu"))
    model_B.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_B.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_B = model_B.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Convert to time series data (use 2022, 2021, 2020, 2019 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_B = model_B.predict(X_2023_seq)

    # Inverse scale
    y_pred_B = scaler_y.inverse_transform(y_pred_scaled_B)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_B = np.sqrt(mean_squared_error(y_test_actual, y_pred_B))
    rmse_list.append(rmse_B)

    # Calculate MAE
    mae_B = mean_absolute_error(y_test_actual, y_pred_B)
    mae_list.append(mae_B)
    
    # Calculate MAPE
    mape_B = mean_absolute_percentage_error(y_test_actual, y_pred_B)
    mape_list.append(mape_B)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_B}, MAE: {mae_B}, MAPE: {mape_B}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
4/4 [==============================] - 24s 1s/step - loss: 0.1007 - accuracy: 0.0000e+00 - val_loss: 0.0460 - val_accuracy: 0.0196
Epoch 2/500
4/4 [==============================] - 0s 32ms/step - loss: 0.0326 - accuracy: 0.0000e+00 - val_loss: 0.0465 - val_accuracy: 0.0196
Epoch 3/500
4/4 [==============================] - 0s 36ms/step - loss: 0.0419 - accuracy: 0.0000e+00 - val_loss: 0.0335 - val_accuracy: 0.0196
Epoch 4/500
4/4 [==============================] - 0s 35ms/step - loss: 0.0265 - accuracy: 0.0000e+00 - val_loss: 0.0352 - val_accuracy: 0.0196
Epoch 5/500
4/4 [==============================] - 0s 32ms/step - loss: 0.0299 - accuracy: 0.0000e+00 - val_loss: 0.0364 - val_accuracy: 0.0196
Epoch 6/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0302 - accuracy: 0.0000e+00 - val_loss: 0.0331 - val_accuracy: 0.0196
Epoch 7/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0261 - accuracy: 0.0000e+00 - val_loss: 0.0322 - val_accuracy: 

In [5]:
# LSTM_C

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_C
    model_C = Sequential()
    model_C.add(Masking(mask_value=0.0, input_shape=(seq_length, X_train.shape[2])))
    model_C.add(LSTM(64, return_sequences=True))
    model_C.add(LSTM(64, return_sequences=True))
    model_C.add(Dropout(rate=0.5))
    model_C.add(TimeDistributed(Dense(10)))
    model_C.add(Flatten())
    model_C.add(Dense(512, activation="relu"))
    model_C.add(Dropout(rate=0.5))
    model_C.add(Dense(64, activation="relu"))
    model_C.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_C.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_C = model_C.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Convert to time series data (use 2022, 2021, 2020, 2019 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_C = model_C.predict(X_2023_seq)

    # Inverse scale
    y_pred_C = scaler_y.inverse_transform(y_pred_scaled_C)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_C = np.sqrt(mean_squared_error(y_test_actual, y_pred_C))
    rmse_list.append(rmse_C)

    # Calculate MAE
    mae_C = mean_absolute_error(y_test_actual, y_pred_C)
    mae_list.append(mae_C)
    
    # Calculate MAPE
    mape_C = mean_absolute_percentage_error(y_test_actual, y_pred_C)
    mape_list.append(mape_C)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_C}, MAE: {mae_C}, MAPE: {mape_C}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
4/4 [==============================] - 52s 3s/step - loss: 0.1174 - accuracy: 0.0000e+00 - val_loss: 0.0465 - val_accuracy: 0.0196
Epoch 2/500
4/4 [==============================] - 0s 38ms/step - loss: 0.0418 - accuracy: 0.0000e+00 - val_loss: 0.0481 - val_accuracy: 0.0196
Epoch 3/500
4/4 [==============================] - 0s 40ms/step - loss: 0.0370 - accuracy: 0.0050 - val_loss: 0.0347 - val_accuracy: 0.0196
Epoch 4/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0309 - accuracy: 0.0000e+00 - val_loss: 0.0349 - val_accuracy: 0.0196
Epoch 5/500
4/4 [==============================] - 0s 40ms/step - loss: 0.0272 - accuracy: 0.0000e+00 - val_loss: 0.0347 - val_accuracy: 0.0196
Epoch 6/500
4/4 [==============================] - 0s 39ms/step - loss: 0.0278 - accuracy: 0.0000e+00 - val_loss: 0.0328 - val_accuracy: 0.0196
Epoch 7/500
4/4 [==============================] - 0s 44ms/step - loss: 0.0249 - accuracy: 0.0000e+00 - val_loss: 0.0315 - val_accuracy: 0.01

In [6]:
# LSTM_D

iterations = 5
rmse_list = []
mae_list = []
mape_list = []

for i in range(iterations):
    # LSTM_D
    model_D = Sequential()
    model_D.add(LSTM(32, input_shape=(seq_length, X_train.shape[2]), return_sequences=True))
    model_D.add(LSTM(32, return_sequences=True))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Flatten())
    model_D.add(Dense(512, activation="relu"))
    model_D.add(Dropout(rate=0.5))
    model_D.add(Dense(64, activation="relu"))
    model_D.add(Dense(1, activation='relu'))

    # Compile
    adam = optimizers.Adam(learning_rate=0.001)
    model_D.compile(loss="mse", optimizer=adam, metrics=["accuracy"])

    # Early stopping callback
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

    # Train the model
    history_D = model_D.fit(X_train, y_train, epochs=500, batch_size=64, validation_split=0.2, verbose=1, callbacks=[early_stopping])

    # Filter 2023 data
    data_23 = common_data[common_data['year'] == 2023]

    # Scale 2023 data
    X_2023_scaled = scaler_X.transform(data_23[features].values)

    # Convert to time series data (use 2022, 2021, 2020, 2019 data to predict 2023)
    def create_sequences_for_prediction(X, seq_length):
        X_seq = []
        for i in range(len(X)):
            seq_x = X[max(0, i - seq_length + 1):i + 1]
            seq_x = np.pad(seq_x, ((seq_length - len(seq_x), 0), (0, 0)), 'constant')
            X_seq.append(seq_x)
        return np.array(X_seq)

    X_2023_seq = create_sequences_for_prediction(X_2023_scaled, seq_length)

    # Predict 2023 data
    y_pred_scaled_D = model_D.predict(X_2023_seq)

    # Inverse scale
    y_pred_D = scaler_y.inverse_transform(y_pred_scaled_D)

    # Restore actual 2023 p_era values for comparison
    y_test_actual = data_23[target].values

    # Calculate RMSE
    rmse_D = np.sqrt(mean_squared_error(y_test_actual, y_pred_D))
    rmse_list.append(rmse_D)

    # Calculate MAE
    mae_D = mean_absolute_error(y_test_actual, y_pred_D)
    mae_list.append(mae_D)
    
    # Calculate MAPE
    mape_D = mean_absolute_percentage_error(y_test_actual, y_pred_D)
    mape_list.append(mape_D)

    print(f'Iteration {i+1}/{iterations} - RMSE: {rmse_D}, MAE: {mae_D}, MAPE: {mape_D}')

# Calculate average RMSE, MAE, MAPE
avg_rmse = np.mean(rmse_list)
avg_mae = np.mean(mae_list)
avg_mape = np.mean(mape_list)

print(f'Average RMSE: {avg_rmse}')
print(f'Average MAE: {avg_mae}')
print(f'Average MAPE: {avg_mape}')

Epoch 1/500
4/4 [==============================] - 14s 1s/step - loss: 0.1019 - accuracy: 0.0000e+00 - val_loss: 0.0313 - val_accuracy: 0.0196
Epoch 2/500
4/4 [==============================] - 0s 61ms/step - loss: 0.0355 - accuracy: 0.0000e+00 - val_loss: 0.0350 - val_accuracy: 0.0196
Epoch 3/500
4/4 [==============================] - 0s 31ms/step - loss: 0.0323 - accuracy: 0.0000e+00 - val_loss: 0.0347 - val_accuracy: 0.0196
Epoch 4/500
4/4 [==============================] - 0s 30ms/step - loss: 0.0344 - accuracy: 0.0000e+00 - val_loss: 0.0362 - val_accuracy: 0.0196
Epoch 5/500
4/4 [==============================] - 0s 32ms/step - loss: 0.0283 - accuracy: 0.0000e+00 - val_loss: 0.0311 - val_accuracy: 0.0196
Epoch 6/500
4/4 [==============================] - 0s 28ms/step - loss: 0.0259 - accuracy: 0.0000e+00 - val_loss: 0.0330 - val_accuracy: 0.0196
Epoch 7/500
4/4 [==============================] - 0s 30ms/step - loss: 0.0264 - accuracy: 0.0000e+00 - val_loss: 0.0313 - val_accuracy: 